In [24]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

# If you have not installed imblearn, run the following line to avoid errors
# !pip install imblearn

# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [25]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
presta_df = pd.read_csv("Resources/lending_data.csv")

# Review the DataFrame
presta_df.head()


,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


In [26]:
#Just curiosity, to check the df columns
presta_df.columns

Index(['loan_size', 'interest_rate', 'borrower_income', 'debt_to_income',
       'num_of_accounts', 'derogatory_marks', 'total_debt', 'loan_status'],
      dtype='object')

### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [27]:
# Separate the data into labels and features by following these two steps:
# 1. Separate the y variable, the labels
y = presta_df['loan_status']

# 2. Separate the X variable, the features. Capital X vs. x it's used as a general convension and dinstiction.
X = presta_df.drop("loan_status", axis=1)

In [29]:
# Review the y variable Series. A view of 6 it's enough

y[:6]


0    0
1    0
2    0
3    0
4    0
5    0
Name: loan_status, dtype: int64

In [30]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [31]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [32]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [33]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', max_iter=250, random_state=1)

# Fit the model using training data
classifier.fit(X_train,y_train)

LogisticRegression(max_iter=250, random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [34]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [35]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, predictions)

0.9520479254722232

In [36]:
# Generate a confusion matrix for the model
training_matrix = confusion_matrix(y_test, predictions)

print(training_matrix)

[[18663   102]
 [   56   563]]


In [37]:
# Print the classification report for the model
testing_report = classification_report(y_test, predictions)
print(testing_report)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** We got a higher scores to identify healthy-loans: Precision: 100%, Recall 99%, and f1-scores 100%. The model has a high percentage of identifying high-risk loans: Precision 85%, Recall 91%, amd f1-score 88%. The high-risk-loans accuracy scores are lower despite the fact than the logistic regression model does a much better job identifying a healthy loan, as per the recall and f1-score values.
    

## Predict a Logistic Regression Model with Resampled Training Data

In [39]:
# Instantiate the random oversampler model
# Assign a random_state parameter of 1 to the model
random_oversampler = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resample, y_resample= random_oversampler.fit_resample(X_train,y_train)

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [40]:
# Count the distinct values of the resampled labels data
y_resample.value_counts()

0    56271
1    56271
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [41]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', max_iter=250,random_state=1)

# Fit the model using the resampled training data
classifier.fit(X_resample,y_resample)


# Make a prediction using the testing data
resample_predictions = classifier.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [42]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, resample_predictions)

0.9936781215845847

In [43]:
# Generate a confusion matrix for the model
resample_training_matrix = confusion_matrix(y_test, resample_predictions)

print(resample_training_matrix)

[[18649   116]
 [    4   615]]


In [44]:
# Print the classification report for the model
resample_testing_report = classification_report(y_test, resample_predictions)
print(resample_testing_report)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.84      0.99      0.91       619

    accuracy                           0.99     19384
   macro avg       0.92      0.99      0.95     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** With the oversampled data, the logistic regression model improved in accuracy scores: Recall 91% to 99%, and f1-score from 88% to 91% despite of getting a lower precision from 85% to 84%.